In [209]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import pandas as pd
import datetime
import math
from typing import Tuple

In [210]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

print(generate_path('bitflyer', 'btcfxjpy', 900, 1675368565))

https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=900&after=1675368565


In [211]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return math.floor(ts)

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883


In [212]:
def get_url(candle: int, date: str, size: int) -> Tuple[str, str]:
        url = 'https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods='
        candle = candle*60
        period = ('%s'%(candle))
        beforeurl = '&before='
        afterurl = '&after='
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        before = str_to_timestamp(date)
        after = before - (candle * size)
        URL=('%s%s%s%s%s%s'%(url, candle, afterurl, after, beforeurl, before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        return URL, period

In [213]:
async def main(candle: int, date: str, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        URL, period = get_url(candle, date, size)

        response = await client.get(URL)
        body = await response.json()
        res = body['result']

        date, op, hi, lo, cl, volume = [], [], [], [], [], []

        for i in res[period]:
            date.append(i[0])
            op.append(i[1])
            hi.append(i[2])
            lo.append(i[3])
            cl.append(i[4])
            volume.append(i[5])

        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime, name='date')
        df = pd.DataFrame({'op' : op, 'hi' : hi, 'lo': lo, 'cl' : cl, 'volume' : volume}, index=dti)
        
        return df

In [214]:
if __name__ == '__main__':
    try:
        df = await main(15, '2023-02-02 20:09:25', 6000)
        display(df)
        print(df.dtypes)
        df.to_pickle('df_ohlcv.pkl')
        df.to_csv('df_bitflyer.csv')
    except KeyboardInterrupt:
        pass

,op,hi,lo,cl,volume
date,,,,,
2022-12-04 22:30:00,2291854,2291941,2289501,2291406,15.380514
2022-12-04 22:45:00,2291456,2291746,2288975,2289177,24.124416
2022-12-04 23:00:00,2289531,2290126,2286200,2290126,33.510044
2022-12-04 23:15:00,2290026,2291511,2289485,2290093,16.977133
2022-12-04 23:30:00,2290286,2290870,2287201,2288617,34.443982
...,...,...,...,...,...
2023-02-02 19:00:00,3076436,3080258,3074061,3079758,38.497302
2023-02-02 19:15:00,3079612,3080461,3076000,3077042,26.473347
2023-02-02 19:30:00,3076671,3078445,3075097,3076117,22.083219


op          int64
hi          int64
lo          int64
cl          int64
volume    float64
dtype: object
